## RAG란?
- 자연어 처리(NLP) 분야의 혁신적인 기술로, 기존 모델의 한계를 넘어서 정보 검색과 생성을 통합하는 방법론
- 풍부한 정보를 담고 있는 대규모 문서 데이터베이스에서 관련정보를 검색하고, 
- 이를 통해 언어 모델이 더 정확하고 상세한 답변을 생성 할 수 있습니다.

### 핵심 개념
- **검색(Retrieval)**: 질문과 관련된 문서나 정보를 벡터 유사도 기반으로 찾기
- **증강(Augmented)**: 찾은 정보를 언어 모델의 컨텍스트에 추가
- **생성(Generation)**: 증강된 정보를 바탕으로 더 정확하고 신뢰할 수 있는 답변 생성


### RAG의 장점
1. 최신 정보 반영 가능 (지식 갱신이 간편)
2. 출처 기반 응답 제공
3. 도메인 특화 응답 가능

### RAG의 기본구조

![RAG.png](./images/RAG.png)



**RAG-FLOW**
![RAG_Flow.png](./images/RAG_Flow.png)


## PDF를 문서로 사용해서 RAG 시스템 구축하기
- 다음 실습 예시는 기본적인 RAG 구조를 구현합니다.
- 추후, 개발을 진행할때는 각각의 구현 상황에 맞는 모듈로 바꾸는 것 만으로도 여러분들만의 RAG 시스템을 구축 할 수 있습니다.

- 예시  
지금 예시에서는 PDF를 읽어와 문서를 저장하고 있지만, 제공되는 문서가 Text일 경우 Docuemnt Loader 모듈을 바꾸어 사용할 수 있습니다.  
또는, 현재는 LLM 모델을 로컬로 Ollama를 이용해 Mi:dm 모델을 사용하고 있지만, API 방식으로 바꾸어 구현 할 수도 있습니다.

**실습자료**
- 제목 : 2025 병 복지 길라잡이  
- 링크 : [국방부 홈페이지 자료위치 링크](https://www.mnd.go.kr/cop/pblictn/selectPublicationUser.do?siteId=mnd&componentId=14&categoryId=0&publicationSeq=1112&pageIndex=1&id=mnd_020704000000)  
- 출처 : 국방부 복지정책과

*링크에서 파일을 다운로드 받은후 data 폴더에 위치시켜주세요*

### Step 1 : 문서 로드(Document Load)

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/Soldier-Benefits-Guide.pdf")

docs = loader.load()

print(f"10번째 문서 내용 : {docs[9].page_content}")

메타데이터 확인해보기

In [ ]:
print(docs[9].__dict__)

### Step 2 : 문서 분할하기 (Text Splitter)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# chunk size : 300자, overlap : 30자
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수: {len(split_documents)}")

### Step 3 : 임베딩 준비하기 (Embedding)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 1. HuggingFaceEmbeddings 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",  # 모델 다운로드
    model_kwargs={'device': 'cpu'},  # CPU 사용 (GPU가 있다면 'cuda'도 가능)
    encode_kwargs={'normalize_embeddings': True}  # 벡터 정규화 활성화
)

print(f"모델명: {embeddings.model_name}")

### Step 4 : Vector Store 생성 및 문서 임베딩

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

### Step 5 : Retriever 생성

In [ ]:
retriever = vectorstore.as_retriever()

### Step 6 : 프롬프트 생성

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문에 답변하는 작업을 수행하는 어시스턴트입니다.
다음에 제공된 문맥 정보를 바탕으로 질문에 답하세요.
정답을 모를 경우, 모른다고만 말하세요.
답변은 반드시 한국어로 작성하세요.

#문맥:
{context}

#질문:
{question}

#답변:"""
)

### Step 7 : LLM 생성

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0,  # 구조화된 출력을 위해 낮은 temperature 설정
)

### Step 8 : 체인 생성

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(), # 다음 체인으로 값을 그대로 넘김
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "병사의 계급별 지급액이 어떻게 돼?"
response = chain.invoke(question)

print(response)